In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

In [ ]:
import dizx
from dizx import Edge
from dizx import clifford_simplifier as simp
Phase = dizx.CliffordPhase
d = 3
Z = dizx.VertexType.Z
X = dizx.VertexType.X

In [ ]:
c = dizx.Circuit(qudit_amount=2,dim=3)
c.add_gates("H H S",0)
# c.add_gate("CZ",0,1)
# c.add_gates("S S H",1)
# c.add_gates("H S", 0)

In [ ]:
print(c)

In [20]:
# Experimenting with sympy's finite fields over polynomials
# Attempt 1 using Polynomial rings

from sympy import symbols, groebner
from sympy.polys.domains import GF, PolynomialRing
from sympy.polys.matrices import DomainMatrix

p = 7
F = GF(p)
beta = symbols("beta")
b = symbols("b")
a = symbols("a")
ainv = symbols("a^{-1}")

# Polynomial ring GF(p)[a]
R = PolynomialRing(F, [a,ainv,b])

Ra = R.convert(a)
Rainv = R.convert(ainv)
# Rbeta = R.convert(beta)
Rb = R.convert(b)

G = groebner([a*ainv - 1], a, ainv, modulus=p)

H = DomainMatrix([[R.convert(0), R.convert(-1)],
                  [R.convert(1), R.convert(0)]],
                 (2, 2), R)

Hinv = DomainMatrix([[R.convert(0), R.convert(1)],
                  [R.convert(-1), R.convert(0)]],
                 (2, 2), R)


def S_repetitions(rep):
    return DomainMatrix([[R.convert(1), R.convert(0)],
                  [rep, R.convert(1)]],
                 (2, 2), R)

# Sbeta = S_repetitions(Rbeta)
Sb = S_repetitions(Rb)
# Build a matrix with entries in GF(p)[a]
A = DomainMatrix([[R.convert(1), R.convert(0)],
                  [R.convert(b), R.convert(1)]],
                 (2, 2), R)

print("Matrix over GF(p)[a]:")
print(A.to_Matrix())

aval = F(3)  # element of GF(7)

# Substitute 'a=3' in each entry
# evaluated_entries = [[entry.evaluate(0,aval) for entry in row]
#                      for row in A.to_list()]

# # Build a new matrix over GF(p)
# A_eval = DomainMatrix(evaluated_entries, A.shape, F)

# print("Matrix evaluated at a=3 (in GF(7)):")
# print(A_eval.to_Matrix())

Matrix over GF(p)[a]:
Matrix([[1, 0], [b, 1]])


In [21]:
# Attempt 2 using groebner bases to automate the reductions of the variables and say a*ainv = 1.
G = groebner([a*ainv - 1], a, ainv, b, modulus=p)
expr = a**2*ainv + 3
G.reduce(expr)

H = Matrix([[0,1],[-1,0]])
Hinv = Matrix([[0,-1],[1,0]])

def Sr(rep):
    return Matrix([[1,0],[rep,1]])

M = H*Sr(-a*(b+1))*Hinv*Sr(-ainv)*Hinv*Sr(-a)*Hinv

def reduce_matrix(M):
    evaluated_entries = []
    for i in range(M.rows):
        row = M.row(i)
        newrow = []
        for entry in row:
            entry = G.reduce(entry)[1]
            newrow.append(entry)
        evaluated_entries.append(newrow)
    return Matrix(evaluated_entries)

reduce_matrix(M)

Matrix([
[     b, -a],
[a^{-1},  0]])

In [16]:
M.row(1)

Matrix([[ainv, a*ainv - 1]])

In [11]:
from sympy import Matrix

M = Matrix([[1,0],[a,1]])
M.inv()

Matrix([
[ 1, 0],
[-a, 1]])

In [ ]:
M = H*Sb*Hinv*Sbeta*Hinv
M

In [ ]:
M*DomainMatrix([[0],[Rb]],(2,1),R)

In [ ]:
Aab = H*S_repetitions(-Ra*(Rb+1))*Hinv*S_repetitions(-Rainv)*Hinv*S_repetitions(-Ra)*Hinv
Aab

In [ ]:
def build_A_matrix(a,b):
    return DomainMatrix([[b, a],
                         [-a, b]],
                         (2, 2), R)

A = build_A_matrix(R.convert(1),R.convert(2))
Omega = DomainMatrix([[R.convert(0), R.convert(1)],
                  [R.convert(-1), R.convert(0)]],
                 (2, 2), R)

A.transpose()*Omega*A

In [ ]:
def eval_matrix(M, values):
    evaluated_entries = []
    for row in M.to_list():
        for entry in row:
            for key,value in values.items():
                entry = entry.evaluate(0)
    [[entry.evaluate() for entry in row]
                      for row in M.to_list()]

In [ ]:
cs = simp.CliffordSimplifier(c)

In [ ]:
cs.dag.children[0].children[0].children

In [ ]:
cs.combine_gates()

In [ ]:
cs.circuit

In [ ]:
cs.dag.children[0].children

In [ ]:
cs.remove_identity_gate()

In [ ]:
cs.push_pauli()

In [ ]:
cs.push_double_hadamard()

In [ ]:
cs.circuit

In [ ]:
cs.dag.children[0].children

In [ ]:
lhs = dizx.Circuit(qudit_amount=1, dim=3)
lhs.add_gates("S X H S H H", 0)
rhs = dizx.Circuit(qudit_amount=1, dim=3)
rhs.add_gates("H S S H", 0)


In [ ]:
c = lhs + rhs.adjoint()
c

In [ ]:
cs = simp.CliffordSimplifier(c)

In [ ]:
cs.simple_optimize()
cs.circuit

In [ ]:
for circ in cs.circuit_list:
    print(circ)

In [ ]:
cs.euler_decomp()

In [ ]:
cs.euler_decomp2()

In [ ]:
cs.dag

In [ ]:
cs.circuit

In [ ]:
cs.simple_optimize()
cs.circuit

In [ ]:
for circ in cs.circuit_list:
    print(circ)

In [ ]:
i = 0
dag = cs.dag.children[0]
while i<10:
    if len(dag.children) == 0: break
    print("Num children: ", len(dag.children))
    print("First child: ", dag.children[0].node)
    dag = dag.children[0]
    i += 1

In [ ]:
cs.push_pauli()
cs.circuit

In [ ]:
cs.combine_gates()
cs.circuit

In [ ]:
cs.remove_identity_gate()
cs.circuit

In [ ]:
cs.circuit

In [ ]:
cs.push_pauli()

In [ ]:
cs.circuit

In [ ]:
cs.push_pauli()

In [ ]:
cs.circuit

In [ ]:
print(cs.push_pauli())
cs.circuit

In [ ]:
print(cs.push_pauli())
cs.circuit

In [ ]:
print(cs.push_pauli())
cs.circuit

In [ ]:
print(cs.combine_gates())
cs.circuit

In [ ]:
print(cs.remove_identity_gate())
cs.circuit